In [1]:
import requests
import json

import pickle
import time
import numpy as np
import pandas as pd
from scipy.stats import t
from termcolor import colored
from sklearn import preprocessing
from sklearn.cluster import Birch
from sklearn.neighbors import KernelDensity
from threading import Thread, Lock

In [4]:
class ESB_Analyzer():
    def __init__(self, esb_data):
        self.esb_data = esb_data
        self.initialize()

    def initialize(self):
        filename = 'birch_model_avgtime.sav'
        self.avg_time_model = pickle.load(open(filename, 'rb'))
        filename = 'birch_model_success.sav'
        self.succee_rate_model = pickle.load(open(filename, 'rb'))
        print('ESB models loaded')

    def update_esb_data(self, esb_data):
        self.esb_data = esb_data

    def birch(self, values, ctype):  # values should be a list
        X = np.reshape(values, (-1, 1))
        brc = self.avg_time_model if ctype == "time" else self.succee_rate_model
        return brc.predict(X)

    def analyze_esb(self, esb_dict):
        esb_tmp = self.esb_data.append(esb_dict, ignore_index=True)
        values = esb_tmp['avg_time'].tolist()
        # print(values)
        birch_labels_time = self.birch(values,"time")
        # birch_labels_rate = self.birch(self.esb_data['avg_time'])
        for label in birch_labels_time:
            if (label != 0):
                print("Found esb_anomaly in avg_time")
                return True

        values = esb_tmp['succee_rate'].tolist()
        # print(values)
        birch_labels_time = self.birch(values,"rate")
        for label in birch_labels_time:
            if (label != 0):
                print("Found esb_anomaly in success rate")
                return True

        self.update_esb_data(esb_tmp)

        return False


class RCA():
    def __init__(self, trace_data, host_data, alpha=0.95, ub=0.02, take_minute_averages_of_trace_data=True, division_milliseconds=60000):
        self.trace_data = trace_data
        self.host_data = host_data
        self.alpha = alpha
        self.ub = ub
        self.anomaly_chart = None
        self.take_minute_averages_of_trace_data = take_minute_averages_of_trace_data
        self.division_milliseconds = division_milliseconds

    def run(self):
        self.trace_processing()
        
        print('Running RCA on %d trace data rows and %d host data rows' %
              (len(self.trace_data), len(self.host_data)))
        overall_start_time = time.time()

        print('Computing for anomaly score chart...')
        self.hesd_trace_detection(alpha=self.alpha, ub=self.ub)
        print('Score chart computed!')

        self.local_initiate()
        output = self.find_anomalous_rows()

        print('The output to send to the server is: ' +
              colored(str(output), 'magenta'))

        print('RCA finished in ' + colored('%f', 'cyan') %
              (time.time() - overall_start_time) + ' seconds.')
        return output

    def esd_test_statistics(self, x, hybrid=True):
        """
        Compute the location and dispersion sample statistics used to carry out the ESD test.
        """
        if hybrid:
            location = pd.Series(x).median(skipna=True) # Median
            dispersion = np.median(np.abs(x - np.median(x))) # Median Absolute Deviation
        else:  
            location = pd.Series(x).mean(skipna=True) # Mean
            dispersion = pd.Series(x).std(skipna=True) # Standard Deviation
            
        return location, dispersion

    def esd_test(self, x, alpha=0.95, ub=0.499, hybrid=True):
        """
        Carries out the Extreme Studentized Deviate(ESD) test which can be used to detect one or more outliers present in the timeseries
        
        x      : List, array, or series containing the time series
        freq   : Int that gives the number of periods per cycle (7 for week, 12 for monthly, etc)
        alpha  : Confidence level in detecting outliers
        ub     : Upper bound on the fraction of datapoints which can be labeled as outliers (<=0.499)
        hybrid : Whether to use the robust statistics (median, median absolute error) or the non-robust versions (mean, standard deviation) to test for anomalies
        """
        nobs = len(x)
        if ub > 0.4999:
            ub = 0.499
        k = max(int(np.floor(ub * nobs)), 1) # Maximum number of anomalies. At least 1 anomaly must be tested.
        #   res_tmp = ts_S_Md_decomposition(x)["residual"] # Residuals from time series decomposition
            
        # Carry out the esd test k times  
        res = np.ma.array(x, mask=False) # The "ma" structure allows masking of values to exclude the elements from any calculation
        anomalies = [] # returns the indices of the found anomalies
        for i in range(1, k+1):
            location, dispersion = self.esd_test_statistics(res, hybrid) # Sample statistics
            tmp = np.abs(res - location) / dispersion
            idx = np.argmax(tmp) # Index of the test statistic
            test_statistic = tmp[idx] 
            n = nobs - res.mask.sum() # sums  nonmasked values
            critical_value = (n - i) * t.ppf(alpha, n - i - 1) / np.sqrt((n - i - 1 + np.power(t.ppf(alpha, n - i - 1), 2)) * (n - i - 1)) 
            if test_statistic > critical_value:
                anomalies.append(test_statistic)
            res.mask[idx] = True
        if len(anomalies) == 0:
            return 0
        return np.mean(anomalies)
    
    def hesd_trace_detection(self, alpha=0.95, ub=0.02):
        grouped_df = self.trace_data.groupby(['cmdb_id', 'serviceName'])[['startTime','actual_time']]

        self.anomaly_chart = pd.DataFrame()
        for (a, b), value in grouped_df:
            value['time_group'] = value.startTime//self.division_milliseconds
            value = value.groupby(['time_group'])['actual_time'].mean().reset_index()
            result = self.esd_test(value['actual_time'].to_numpy(), alpha=alpha, ub=ub, hybrid=True)
            self.anomaly_chart.loc[b,a] = result

        self.anomaly_chart = self.anomaly_chart.sort_index()
        print(self.anomaly_chart)
        # print(self.anomaly_chart.to_dict())
        return self.anomaly_chart
    
    def local_initiate(self):
        self.dockers = ['docker_001', 'docker_002', 'docker_003', 'docker_004',
                'docker_005', 'docker_006', 'docker_007', 'docker_008']
        self.docker_hosts = ['os_017', 'os_018', 'os_019', 'os_020']

        self.docker_kpi_names = ['container_cpu_used', None]
        self.os_kpi_names = ['Sent_queue', 'Received_queue']
        self.db_kpi_names = ['Proc_User_Used_Pct','Proc_Used_Pct','Sess_Connect','On_Off_State', 'tnsping_result_time']

        self.docker_lookup_table = {}
        for i in range(len(self.dockers)):
            self.docker_lookup_table[self.dockers[i]] = self.docker_hosts[i % 4]


    def find_anomalous_rows(self, min_threshold = 5):
        table = self.anomaly_chart.copy()
        threshold = max( 0.5 * table.stack().max(), min_threshold)
        dodgy_rows = []
        just_rows = []
        row_col_dict = {}
        for column in table:
            v = 0
            r = ''
            for index, row in table.iterrows():
                if (row[column] > threshold):
                    if index == column:
                        dodgy_rows.append([index, column, row[column]])
                        just_rows.append(index)
                        row_col_dict[index] = True
                        break
                    elif (row[column] > v):
                        v = row[column]
                        r = index
            if r != '':
                dodgy_rows.append([r, column, v])
                just_rows.append(r)
                if r not in row_col_dict.keys():
                    row_col_dict[r] = False
        output = self.localize(dodgy_rows, list(set(just_rows)), row_col_dict)
        return output


    def find_anomalous_kpi(self, cmdb_id, row_col_same = False):
        kpi_names = []
        if 'os' in cmdb_id:
            kpi_names = self.os_kpi_names
        elif 'docker' in cmdb_id:
            kpi_names = [self.docker_kpi_names[0]] if row_col_same else [self.docker_kpi_names[1]]
        else:
            kpi_names = self.db_kpi_names
            host_data_subset = self.host_data.loc[(self.host_data.cmdb_id == cmdb_id) & (self.host_data.name.isin(kpi_names))]
            results_dict = {}
            for kpi, values in host_data_subset.groupby('name')['value']:
                values = list(values)
                score =  self.esd_test(np.array(values), 0.95, 0.1, True)
                results_dict[kpi] = score
            db_connection_limit = [results_dict['Proc_User_Used_Pct'],results_dict['Proc_Used_Pct'],results_dict['Sess_Connect']]
            db_connection_limit_score = np.mean(db_connection_limit)
            db_close = [results_dict['On_Off_State'],results_dict['tnsping_result_time']]
            db_close_score = np.mean(db_close)
            if db_connection_limit_score > db_close_score:
                kpi_names = kpi_names[:3]
            else:
                kpi_names = kpi_names[3:]             

        return kpi_names


    def localize(self, dodgy_rows, just_rows, row_col_dict):
        n = len(just_rows)
        print('%d anomalies found' % n)
        if n < 1:
            return None
        if n == 1:
            KPIs = self.find_anomalous_kpi(just_rows[0], row_col_dict[list(row_col_dict.keys())[0]])
            to_be_sent = []
            for KPI in KPIs:
                to_be_sent.append([just_rows[0], KPI])
            return to_be_sent
        if n == 2:
            r0 = just_rows[0]
            r1 = just_rows[1]
            if ('os' in r0) and ('os' in r1):
                KPIS = self.find_anomalous_kpi('os_001')
                return [['os_001', KPIS[0]], ['os_001', KPIS[1]]]

            if ('docker' in r0) and ('docker' in r1):
                if self.docker_lookup_table[r0] == self.docker_lookup_table[r1]:
                    KPIS = self.find_anomalous_kpi(self.docker_lookup_table[r0])
                    to_be_sent = []
                    for kpi in KPIS:
                        to_be_sent.append([self.docker_lookup_table[r0], kpi])
                    return to_be_sent

            KPI0s = self.find_anomalous_kpi(r0, row_col_dict[r0])
            KPI1s = self.find_anomalous_kpi(r1, row_col_dict[r1])
            to_be_sent = []
            for kpi in KPI0s:
                to_be_sent.append([r0, kpi])
            for kpi in KPI1s:
                to_be_sent.append([r1, kpi])
            return to_be_sent
        if n > 2:
            dodgy_rows.sort(key = lambda x: x[2], reverse = True)
            just_rows = [x[0] for x in dodgy_rows]
            just_rows = list(np.unique(just_rows))
            row_col_dict = { just_rows[0]: row_col_dict[just_rows[0]], just_rows[1]: row_col_dict[just_rows[1]] } 
            return self.localize(dodgy_rows[:2], just_rows[:2], row_col_dict)


    def update_trace_data(self, trace_data):
        self.trace_data = trace_data

    def update_host_data(self, host_data):
        self.host_data = host_data

    def trace_processing(self):
        print("Started trace processing")
        p_time = time.time()
        self.trace_data = self.trace_data[self.trace_data['callType'] != 'FlyRemote'].copy()
        df1 = self.trace_data[self.trace_data['callType']=='RemoteProcess']
        df1 = df1[['pid','cmdb_id']]
        df1 = df1.set_index('pid')

        csf_cmdb = df1.to_dict()
        csf_cmdb = {str(key):str(values) for key, values in csf_cmdb['cmdb_id'].items()}

        # for index, row in self.trace_data.iterrows():
        #     if row['id'] in csf_cmdb:
        #         self.trace_data.at[index, 'serviceName'] = csf_cmdb[row['id']]

        elapse_time = {}
        children = {}

        def do_thing(row):
            ## if change 297 and 298, gets different result??? loook into this plz
            if row['id'] in csf_cmdb:
                row['serviceName'] = csf_cmdb[row['id']]
            if row['pid'] != 'None':
                children[row['pid']] = children.get(row['pid'], [])
                children[row['pid']].append(row['id'])
            elapse_time[row['id']] = float(row['elapsedTime'])
            return row
        self.trace_data = self.trace_data.apply(do_thing, axis=1)


        # for index, row in self.trace_data.iterrows():
        #     if row['pid'] != 'None':
        #         if row['pid'] in children.keys():
        #             children[row['pid']].append(row['id'])
        #         else:
        #             children[row['pid']] = [row['id']]
        #     elapse_time[row['id']] = float(row['elapsedTime'])
        
        # self.trace_data.apply(parse, axis = 1)

        self.trace_data['actual_time'] = 0.0
        # for index, row in self.trace_data.iterrows():
        #     total_child = 0.0
        #     if row['id'] not in children.keys():
        #         self.trace_data.at[index, 'actual_time'] = row['elapsedTime']
        #         continue
        #     for child in children[row['id']]:
        #         total_child += elapse_time[child]
        #     self.trace_data.at[index, 'actual_time'] = row['elapsedTime'] - total_child

        def get_actual_time(row):
            total_child = 0.0
            if row['id'] in children:
                for child in children[row['id']]:
                    total_child += elapse_time[child]
            row['actual_time'] = row['elapsedTime'] - total_child
            return row
        
        self.trace_data = self.trace_data.apply(get_actual_time, axis = 1)
        
        self.trace_data = self.trace_data[~(self.trace_data['serviceName'].str.contains('csf', na=True))]

        print("Trace processed in ", time.time()-p_time, 'seconds')
        print(self.trace_data)


# Three topics are available: platform-index, business-index, trace.
# Subscribe at least one of them.
AVAILABLE_TOPICS = set(['platform-index', 'business-index', 'trace'])
# CONSUMER = KafkaConsumer('platform-index', 'business-index', 'trace',
#                          bootstrap_servers=['172.21.0.8', ],
#                          auto_offset_reset='latest',
#                          enable_auto_commit=False,
#                          security_protocol='PLAINTEXT')


class Trace():  # pylint: disable=invalid-name,too-many-instance-attributes,too-few-public-methods
    '''Structure for traces'''

    __slots__ = ['call_type', 'start_time', 'elapsed_time', 'success',
                 'trace_id', 'id', 'pid', 'cmdb_id', 'service_name', 'ds_name']

    def __new__(self, data):
        self.trace = data
        if self.trace['callType'] == 'JDBC' or self.trace['callType']=='LOCAL':
            try:
                self.trace['serviceName'] = data['dsName']
            except:
                print(data)
                print('JDBC doesnt have dsName')
        
        elif self.trace['callType']=='RemoteProcess' or self.trace['callType']=='OSB':
            self.trace['serviceName'] = data['cmdb_id']

        if 'dsName' in self.trace:
            self.trace.pop('dsName')

        return self.trace


def detection(timestamp):
    global host_df, trace_df
    print('Starting Anomaly Detection')
    startTime = timestamp - 1200000  # one minute before anomaly

    # print(len(trace_df), trace_df.head())
    # print(len(host_df), host_df.head())
    trace_df_temp = trace_df[(trace_df['startTime'] >= startTime) &
                             (trace_df['startTime'] <= timestamp)]
    host_df_temp = host_df[(host_df['timestamp'] >= startTime) &
                           (host_df['timestamp'] <= timestamp)]
    print(len(trace_df_temp), trace_df_temp.head())
    print(len(host_df_temp), host_df_temp.head())

    rca_temp = RCA(trace_data=trace_df_temp, host_data=host_df_temp)
    results_to_send_off = rca_temp.run()

    print('Anomaly Detection Done.')
    if results_to_send_off is None:
        # print('Nothing detected')
        return False, rca_temp
    # for a in anom_hosts:
    #     item = a.split(':')[0]
    #     if (item not in anoms):
    #         anoms.append(item)
    # print(results_to_send_off)
    # submit(results_to_send_off)
    return True, rca_temp


def rcaprocess(esb_item, trace, host, timestamp, lock):
    global host_df, trace_df, esb_anal, a_time
    esb_anomaly = False

    # print(trace)
    trace_df = trace_df[(trace_df.startTime >= (timestamp-1260000))]
    host_df = host_df[(host_df.timestamp >= (timestamp-1260000))]
    
    t = time.time()
    t_df = pd.DataFrame(trace)
    h_df = pd.DataFrame(host)

    trace_df = pd.concat([trace_df, t_df], axis=0, ignore_index=True)
    host_df = pd.concat([host_df, h_df], axis=0, ignore_index=True)

    print('Time to add new data: ', (time.time()-t))

    print(esb_anal.esb_data.tail(1))
    print(host_df.tail(1))
    print(trace_df.tail(1))

    with lock:
        esb_anomaly = esb_anal.analyze_esb(esb_item)
        if (time.time() - a_time) >= 600 and esb_anomaly:
            tmp_time = time.time()
            print("oops")
            # detection(timestamp)
            result = detection(timestamp)
            print('Anomaly at: ', timestamp)
            if result:
                a_time = tmp_time


In [417]:
global host_df, trace_df

path = 'training_data/'
trace_df = pd.read_csv(path + 'trace_data_os17_new.csv')
# trace_df = trace_df.drop(['actual_time','path'], axis=1)
# trace_df = trace_df.drop(['path'], axis=1)
trace_df = trace_df.sort_values(by=['startTime'], ignore_index=True)
# trace = trace[trace.startTime < trace.startTime[0]+1260000]

host_df = pd.read_csv(path + 'kpi_data_os17_new.csv')
host_df = host_df.sort_values(by=['timestamp'], ignore_index=True)

# print(trace_df)
print(host_df)
timestamp = int(host_df['timestamp'].iloc[-1]-180000)
# print(timestamp)
# trace_df = trace_df[(trace_df.startTime >= (timestamp-1260000))]
# print(host_df)
boo, rca = detection(timestamp)


       Unnamed: 0           itemid                      name     bomc_id  \
0          173549         63309180  Incoming_network_traffic  ZJ-001-069   
1          521522  999999998651100             MEM_real_util  ZJ-002-053   
2          521523  999999998651280              CPU_free_pct  ZJ-002-056   
3          521524  999999998650680              CPU_free_pct  ZJ-002-056   
4          521525  999999998650980              CPU_free_pct  ZJ-002-056   
...           ...              ...                       ...         ...   
41601      194578         65439900              Disk_io_util  ZJ-001-125   
41602      194579         63308940  Outgoing_network_traffic  ZJ-001-072   
41603      194582  999999996431220             FS_used_space  ZJ-001-108   
41604      194580  999999996430740                Send_total  ZJ-001-074   
41605      533670  999999998651100             MEM_real_util  ZJ-002-053   

           timestamp      value cmdb_id  
0      1590348420000   0.000000  os_015  
1  

In [587]:
table = pd.DataFrame({'docker_001': {'db_003': np.nan, 'db_007': 2.9813413195424343, 'db_009': 3.0209533010263447, 'docker_001': 4.853138514249491, 'docker_002': np.nan, 'docker_003': np.nan, 'docker_004': np.nan, 'docker_005': 3.551310136788956, 'docker_006': 6.0325483894159975, 'docker_007': np.nan, 'docker_008': np.nan, 'os_021': np.nan, 'os_022': np.nan}, 'docker_002': {'db_003': np.nan, 'db_007': 3.3295248353546856, 'db_009': 1.941661540366842, 'docker_001': np.nan, 'docker_002': 2.22147986919222, 'docker_003': np.nan, 'docker_004': np.nan, 'docker_005': 3.4232516837614972, 'docker_006': 2.921920937021125, 'docker_007': np.nan, 'docker_008': np.nan, 'os_021': np.nan, 'os_022': np.nan}, 'docker_003': {'db_003': np.nan, 'db_007': 4.505818517747893, 'db_009': 4.2944222984892315, 'docker_001': np.nan, 'docker_002': np.nan, 'docker_003': 3.4296830945060464, 'docker_004': np.nan, 'docker_005': np.nan, 'docker_006': np.nan, 'docker_007': 4.127451447424727, 'docker_008': 4.260659945792008, 'os_021': np.nan, 'os_022': np.nan}, 'docker_004': {'db_003': np.nan, 'db_007': 4.367334230877272, 'db_009': 2.5614473718663886, 'docker_001': np.nan, 'docker_002': np.nan, 'docker_003': np.nan, 'docker_004': 2.5347650585408923, 'docker_005': np.nan, 'docker_006': np.nan, 'docker_007': 3.463050917628577, 'docker_008': 2.8145334777748636, 'os_021': np.nan, 'os_022': np.nan}, 'docker_005': {'db_003': 4.21441011069078, 'db_007': np.nan, 'db_009': np.nan, 'docker_001': np.nan, 'docker_002': np.nan, 'docker_003': np.nan, 'docker_004': np.nan, 'docker_005': 3.6864016848501717, 'docker_006': np.nan, 'docker_007': np.nan, 'docker_008': np.nan, 'os_021': np.nan, 'os_022': np.nan}, 'docker_006': {'db_003': 4.970411934042088, 'db_007': np.nan, 'db_009': np.nan, 'docker_001': np.nan, 'docker_002': np.nan, 'docker_003': np.nan, 'docker_004': np.nan, 'docker_005': np.nan, 'docker_006': 7.576686437915655, 'docker_007': np.nan, 'docker_008': np.nan, 'os_021': np.nan, 'os_022': np.nan}, 'docker_007': {'db_003': 7.587337997134692, 'db_007': np.nan, 'db_009': np.nan, 'docker_001': np.nan, 'docker_002': np.nan, 'docker_003': np.nan, 'docker_004': np.nan, 'docker_005': np.nan, 'docker_006': np.nan, 'docker_007': 6.948016248094616, 'docker_008': np.nan, 'os_021': np.nan, 'os_022': np.nan}, 'docker_008': {'db_003': 9.80294723945619, 'db_007': np.nan, 'db_009': np.nan, 'docker_001': np.nan, 'docker_002': np.nan, 'docker_003': np.nan, 'docker_004': np.nan, 'docker_005': np.nan, 'docker_006': np.nan, 'docker_007': np.nan, 'docker_008': 2.4692037430959335, 'os_021': np.nan, 'os_022': np.nan}, 'os_021': {'db_003': np.nan, 'db_007': np.nan, 'db_009': np.nan, 'docker_001': np.nan, 'docker_002': np.nan, 'docker_003': 4.007029991190557, 'docker_004': 4.752495280194077, 'docker_005': np.nan, 'docker_006': np.nan, 'docker_007': np.nan, 'docker_008': np.nan, 'os_021': 4.3734588741685645, 'os_022': np.nan}, 'os_022': {'db_003': np.nan, 'db_007': np.nan, 'db_009': np.nan, 'docker_001': 7.398256781684731, 'docker_002': 9.576412954443716, 'docker_003': np.nan, 'docker_004': np.nan, 'docker_005': np.nan, 'docker_006': np.nan, 'docker_007': np.nan, 'docker_008': np.nan, 'os_021': np.nan, 'os_022': 92.44926368399037}}
)
table

,docker_001,docker_002,docker_003,docker_004,docker_005,docker_006,docker_007,docker_008,os_021,os_022
db_003,NaN,NaN,NaN,NaN,4.214410,4.970412,7.587338,9.802947,NaN,NaN
db_007,2.981341,3.329525,4.505819,4.367334,NaN,NaN,NaN,NaN,NaN,NaN
db_009,3.020953,1.941662,4.294422,2.561447,NaN,NaN,NaN,NaN,NaN,NaN
docker_001,4.853139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.398257
docker_002,NaN,2.221480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.576413
docker_003,NaN,NaN,3.429683,NaN,NaN,NaN,NaN,NaN,4.007030,NaN
docker_004,NaN,NaN,NaN,2.534765,NaN,NaN,NaN,NaN,4.752495,NaN
docker_005,3.551310,3.423252,NaN,NaN,3.686402,NaN,NaN,NaN,NaN,NaN
docker_006,6.032548,2.921921,NaN,NaN,NaN,7.576686,NaN,NaN,NaN,NaN
docker_007,NaN,NaN,4.127451,3.463051,NaN,NaN,6.948016,NaN,NaN,NaN


In [588]:
threshold = max( 0.5 * table.stack().max(), 5)
dodgy_rows = []
just_rows = []
row_col_dict = {}
for column in table:
    v = 0
    r = ''
    for index, row in table.iterrows():
        if (row[column] > threshold):
            if index == column:
                dodgy_rows.append([index, column, row[column]])
                just_rows.append(index)
                row_col_dict[index] = True
                break
            elif (row[column] > v):
                v = row[column]
                r = index
    if r != '':
        dodgy_rows.append([r, column, v])
        just_rows.append(r)
        if r not in row_col_dict.keys():
            row_col_dict[r] = False
# output = self.localize(dodgy_rows, list(set(just_rows)), row_col_dict)
print(list(set(just_rows)))
dodgy_rows

['os_022']


[['os_022', 'os_022', 92.44926368399037]]

In [589]:
import operator
values = sorted(table.stack().tolist())

from sklearn.ensemble import IsolationForest
# identify outliers in the training dataset
X = np.reshape(values, (-1, 1))
iso = IsolationForest()
yhat = iso.fit_predict(X)
print('\nyhat ', yhat)

import statistics

thresh_idx = 0
seen_normal = False
for i in range(len(yhat)):
    if (yhat[i] == 1):
        seen_normal = True
    elif (yhat[i] == -1) and seen_normal:
        thresh_idx = i
        break


secondary = values[thresh_idx:-1]
print('\nsec ', secondary)
med = statistics.median(secondary)
threshold = statistics.median([abs(x-med) for x in secondary])+secondary[0]
# threshold= secondary[0]

print('\nthresh ', threshold)

row_col_dict = {}
column_dict= {}
row_dict = {}

for column in table:
    for index, row in table.iterrows():
       if (str(row[column]) != 'nan'):
           if (column in column_dict.keys()):
               column_dict[column]= (column_dict[column]+ row[column])/2
           else:
               column_dict[column]= row[column]

           if (index in row_dict.keys()):
               row_dict[index]= (column_dict[column]+ row[column])/2
           else:
               row_dict[index]=row[column]

           if index == column:
               row_col_dict[index] = True
           else:
               if index not in row_col_dict.keys():
                   row_col_dict[index] = False

print('\nrow col dict', row_col_dict)

for key in row_dict.keys():
    if (key in column_dict.keys()):
        row_dict[key] = (row_dict[key]+ column_dict[key]) //2
print('\nrow dict', row_dict)
print()

just_rows =  {v:k for k, v in sorted(row_dict.items(), key=operator.itemgetter(1),reverse=True)}

row_vals = list(just_rows.keys())
X = np.reshape(row_vals, (-1, 1))
iso = IsolationForest()
yhat = iso.fit_predict(X)
print('\nyhat ', yhat)

output = []
for i in range(len(yhat)):
    if (yhat[i] == -1):
        output.append(just_rows[row_vals[i]])
output


yhat  [-1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1 -1 -1 -1 -1 -1 -1 -1 -1]

sec  [6.0325483894159975, 6.948016248094616, 7.398256781684731, 7.576686437915655, 7.587337997134692, 9.576412954443716, 9.80294723945619]

thresh  6.661218579237037

row col dict {'db_007': False, 'db_009': False, 'docker_001': True, 'docker_005': True, 'docker_006': True, 'docker_002': True, 'docker_003': True, 'docker_007': True, 'docker_008': True, 'docker_004': True, 'db_003': False, 'os_021': True, 'os_022': True}

row dict {'db_007': 4.367334230877272, 'db_009': 3.0129190866191093, 'docker_001': 6.0, 'docker_005': 3.0, 'docker_006': 6.0, 'docker_002': 5.0, 'docker_003': 4.0, 'docker_007': 7.0, 'docker_008': 5.0, 'docker_004': 3.0, 'db_003': 9.80294723945619, 'os_021': 4.0, 'os_022': 71.0}


yhat  [-1 -1  1  1  1  1  1  1  1]


['os_022', 'db_003']

['docker_002']